### **_Libraries_**

In [5]:
# Keys
from __Alpaca_API_Keys__ import *

# Trading
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.requests import GetOrdersRequest
from alpaca.trading.enums import OrderSide
from alpaca.trading.enums import OrderType
from alpaca.trading.enums import TimeInForce

# Time 
from datetime import datetime
import threading
import time
import pytz

# Data
import numpy as np
import pandas as pd
import yfinance as yf

# Web-Scraping
from bs4 import BeautifulSoup as bs
import requests

# Recording
import plotly.graph_objects as go
import logging

In [ ]:
logging.basicConfig(level=logging.INFO,
                    filename="c:/users/redmo/Desktop/Trading/Logs/First Test",
                    filemode="a",
                    format="%(asctime)s - %(message)s", datefmt='%d-%b-%y %H:%M:%S')

client = TradingClient(API_KEY, SECRET_KEY)

### **_Trading Class and Threader Function_**

In [18]:
class Trading:
    """
    ## The Trading Class

    ### Description:
    This class allows the user to create multiple trading "bots" based on a
    chosen strategy with optional parameters. These bots can then be run
    concurrently using threading. The bots place trades on the Alpaca website
    using the Alpaca API.

    You can also inspect the trades made by the bot by using the plotting 
    method which gives a detailed view of all the trades in relation to the
    price and various indicators.

    If you want to change the strategy used, you would have to physically
    change the code in the Data and Strategy functions within the Bot method.
    You might also have to change the Orders function, the logging messages,
    the status prints, the Features method and the Plotting method too as it 
    only has BBands indicators by default.

    ### Args:
        - client (Alpaca trading client): Trading client linking to your Alpaca account
        - symbol (string): The stock symbol you wish to trade on
        - moving_average (int): The moving average window
        - standard_deviation (float): The coefficient for the standard deviation
        - quantity (int): The number of stocks you want to buy or sell at a time
        - timeframe (int): Your chosen trading timeframe in seconds

    ### Methods:
        - Features: Prints out all attributes and details about your trading object
        - Bot: Runs a chosen strategy based on given parameters
        - Plotting: Returns a plot of all trades made while your object was running
        
    ### Notes:
    The default strategy used in the Bot method is Bollinger Bands. This
    must be changed manually. The Plotting method is also built around a
    Bollinger Bands strategy so this also must be considered if you wish
    to change the strategy.
    """
    
    def __init__(self, client, symbol, moving_average, standard_deviation, quantity, timeframe):
        # Setting initial attributes of the class.
        self.cl = client
        self.sb = symbol
        self.ma = moving_average
        self.sd = standard_deviation
        self.qt = quantity
        self.tf = timeframe
        print("Initialised")

    def Features(self):
        """
        ### Features
        A method to print all current attributes and details about the bot
        you've created.

        ### Args:
            - self (instance): Instance of the class
        
        ### Returns:
            - None
        """
        # Basic stats
        print(f"Client:               {self.cl}")
        print(f"Symbol:               {self.sb}")
        print(f"Quantity:             {self.qt}")
        print(f"Timeframe:            {self.tf}s")

        # Optional stats as these depend on the chosen strategy
        print(f"Moving Average:       {self.ma}")
        print(f"Standard Deviation:   {self.sd}")

    def Bot(self, stop_time, cancel, logger):
        """
        ### Bot
        A method to run a strategy which places trades using the Alpaca API.
        First, close data is webscraped from Yahoo Finance. It's then ordered
        into a dataframe. A specific strategy is then ran on the data. The 
        trades are then recorded in the History function.

        ### Args: 
            - self (instance): Instance of the class
            - stop_time (string): A structured datetime string in the format "Y-m-d H:M:S"
            - messages (bool): A Bool which determines if certain status messages are printed to the screen
            - logger (module): Logger object to monitor script status

        ### Returns:
            - combined (df): Pandas dataframe of all close values collected while running
            - trades (df): Pandas dataframe of all trades made while running
        """

        # Start time
        current = datetime.now()
        year, month, day = current.year, current.month, current.day
        hour, minute, second = current.hour, current.minute, current.second
        start_time = f"{year}-{month}-{day} {hour}:{minute}:{second}"
        self.bg = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')

        # Match it to the timezone
        tz = pytz.timezone("UTC")
        self.bg = tz.localize(self.bg)

        # Stop time, cancel positions and logger
        stop_time = datetime.strptime(stop_time, '%Y-%m-%d %H:%M:%S')
        self.st = stop_time
        self.cp = cancel
        self.lg = logger

        # Functions for the bot
        def Time(self):
            """
            A function to calculate the time difference from the current time 
            and the given stop time.

            Args:
                self (instance): Instance of the class

            Returns:
                dif (float): Difference between current time and stop time in seconds
            """
            now = datetime.now()
            dif = (self.st - now).total_seconds()
            return dif

        def Sleep(self):
            """
            A function that calculates number of seconds until the next minute.

            Args:
                self (instance): Instance of the class

            Returns:
                sleep (float): Number of seconds until next minute
            """
            current = time.time()
            sleep = self.tf - (current % self.tf)
            return sleep
        
        def Webscraping(self):
            """
            A function to webscrape real-time close data from the Yahoo Finance website.

            Args:
                self (instance): Instance of the class

            Returns:
                price (float): Real-time close price
            """
            # Initialising vars to access Yahoo Finance
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
            url = f"https://finance.yahoo.com/quote/{self.sb}"
            
            # Extracting the HTML code
            request = requests.get(url, headers = headers)
            self.lg.info("WS: Request made")
            htmlcode = bs(request.text, "html.parser")
            self.lg.info("WS: Code gathered")

            # Parsing the code to find the current price 
            price = float(htmlcode.find("fin-streamer", {"class": "Fw(b) Fz(36px) Mb(-4px) D(ib)"}).text.replace(',', ''))
            self.lg.info(f"WS: Price created {price}")

            return price
        
        def Data(self, price, previous):
            """
            A function that takes the previous webscraped data and most recent addition and combines
            them. The data is then used to create specific features and parameters used in the chosen
            strategy.

            Args:
                self (instance): Instance of the class
                price (float): Real-time close price
                previous (df): All price data and specific features accumulated while running
                
            Returns:
                combined (df): "previous" df combined with new price data. Becomes "previous" df in
                next iteration
            """
            # Getting current time info for the dataframe index
            current = datetime.now()
            year, month, day = current.year, current.month, current.day
            hour, minute, second = current.hour, current.minute, current.second
            date = f"{year}-{month}-{day} {hour}:{minute}:{second}"
            self.lg.info(f"DT: Date created {date}")

            # Assembling the price and feature dataframe
            dataframe = pd.DataFrame({"Price": price}, index=[date])
            combined = pd.concat([previous, dataframe])
            self.lg.info("DT: Combined created")

            # Creating feature columns for the chosen strategy
            combined["Moving Average"] = combined["Price"].rolling(self.ma).mean()
            combined["Standard Deviation"] = combined["Price"].rolling(self.ma).std()
            combined["Upper Band"] = combined["Moving Average"] + (self.sd * combined["Standard Deviation"])
            combined["Lower Band"] = combined["Moving Average"] - (self.sd * combined["Standard Deviation"])

            # Setting the index to a Datetime format
            combined.index = pd.to_datetime(combined.index)
            self.lg.info(f"DT: Features created")

            return combined
        
        def Orders(self, side):
            """
            A function to send orders to the Alpaca website.

            Args:
                self (instance): Instance of the class
                side (OrderSide): order to buy or sell
            """
            # Order bracket
            order_data = MarketOrderRequest(
                symbol=self.sb,
                qty=self.qt,
                side=side,
                type=OrderType.MARKET,
                time_in_force=TimeInForce.GTC)
            self.lg.info("OD: Order Bracket created")

            # Submitting the order bracket
            self.cl.submit_order(order_data=order_data)
            self.lg.info("OD: Order submitted")

        def Strategy(self, combined):
            """
            A function to check whether the strategy's conditions have been met. If so,
            an order is sent using the "Orders" function.

            Args:
                self (instance): Instance of the class
                combined (df): Df of all strategy features
            """
            # Condition 1: If the close price is above the upperband, Sell
            if combined["Price"].iloc[-1] > combined["Upper Band"].iloc[-1]:
                Orders(self, OrderSide.SELL)
                print("Sold")
                self.lg.info("ST: Sell")
            
            # Condition 2: If the close price is below the lowerband, Buy
            elif combined["Price"].iloc[-1] < combined["Lower Band"].iloc[-1]:
                Orders(self, OrderSide.BUY)
                print("Bought")
                self.lg.info("ST: Buy")
        
        def History(self, previous):
            """
            A function to gather and record trades made while running

            Args:
                self (instance): Instance of the class
                previous (df): All trades accumulated and recorded while running

            Returns:
                trades (df): Combined dataframe of the most recent trade and all 
                preious trades recorded while running 
            """
            # Getting 50 most recent orders
            orders = self.cl.get_orders(filter=GetOrdersRequest(status="closed", symbols=[self.sb]))
            self.lg.info("HT: Order data gathered")

            # Extracting specifics from the most recent trade
            side = orders[0].side
            time = orders[0].filled_at
            fill = orders[0].filled_avg_price
            self.lg.info("HT: Order specifics gathered")

            # Converting data type of the fill price to a float
            if fill is not None:
                self.lg.info(f"HT: Fill is a number: {fill}")
                fill = float(fill)
            else:
                self.lg.info(f"HT: Fill is None: {fill}")
                fill = None

            # Creating data for plotting based on the trade
            if side == OrderSide.BUY:
                colour = "green"
                marker = "triangle-up"
            elif side == OrderSide.SELL:
                colour = "red"
                marker = "triangle-down"

            # Creating a dataframe to store data abou the trade
            trade = pd.DataFrame(dict({"Avg Fill Price":fill, "Side":side, "Colour":colour, "Marker":marker}), index=[time])
            self.lg.info("HT: Trade dataframe created")

            # Combining the previously recorded trades and most recent trade
            trades = pd.concat([previous, trade])
            self.lg.info("HT: Trade dataframe combined")

            # If there are no new trades, a duplicate is made so it's gotten rid of here
            trades = trades.drop_duplicates(keep='last')
            self.lg.info("HT: Duplicates removed")

            return trades

        # Establishing the finish time and placeholders
        finish = (time.time() + Time(self))
        combined = pd.DataFrame()
        trades = pd.DataFrame()

        self.lg.info("Bot Initialised")

        # Loop until stoptime
        while time.time() < finish:

            # Sleeping until next increment in chosen timeframe
            time.sleep(Sleep(self))

            # Webscraping close data from Yahoo Finance website
            try:
                price = Webscraping(self)
            except Exception as e:
                logger.error(f"Webscraping ERROR: {e}", exc_info=True)
                print("Webscraping ERROR:", e)

            # Collecting webscraped close data and creting features for chosen strategy
            try:
                combined = Data(self, price, combined)
            except Exception as e:
                logger.error(f"Data ERROR: {e}", exc_info=True)
                print("Data ERROR:", e)

            # Submitting orders if conditions are met from the calculated features 
            try:
                Strategy(self, combined)
            except Exception as e:
                logger.error(f"Strategy ERROR: {e}", exc_info=True)
                print("Strategy ERROR:", e)

            # Recording trades if any are made
            try:
                trades = History(self, trades)
            except Exception as e:
                logger.error(f"History ERROR: {e}", exc_info=True)
                print("History ERROR:", e, )

        # Assigning new data and trades to attributes to be accessed easily later on
        self.cb = combined
        self.tr = trades

        if self.cp == True:
            self.cl.close_all_positions(cancel_orders = True)

    def Plotting(self, interval):
        """
        ### Plotting
        A method to disply the trades made throughout the day and chosen
        indicators. Must change the indicators yourself however.

        ### Args:
            - self (instance): Instance of the class
            - interval (string): OHLC timeframe can only be the following:
            [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]

        ### Returns:
            - None
        """
        # Gathering real OHLC data from Yahoo Finance
        ohlc = yf.Ticker(self.sb)
        ohlc = ohlc.history(period="5d", interval=interval)  
        ohlc.index = ohlc.index.tz_convert("UTC")
        ohlc = ohlc.loc[self.bg:]

        # Removes that 1 trade which was made before running
        trades = self.tr.loc[self.bg:]

        # Clears out flat period before the market opens
        market_open = f"{self.bg.year}-{self.bg.month}-{self.bg.day} {14}:{29}:{00}"
        indicators = self.cb.loc[market_open:]

        # Creating the figure for plotting
        fig = go.Figure()

        # Plotting the trades using a scatter
        fig.add_trace(go.Scatter(x=np.array(trades.index),
                                 y=trades["Avg Fill Price"],
                                 mode="markers",
                                 name="Buys + Sells",
                                 marker=dict(color=trades["Colour"], size=15, symbol=trades["Marker"])))
        
        # Plotting the close prices we collected by webscraping using a line plot
        fig.add_trace(go.Scatter(x=np.array(indicators.index),
                                 y=indicators["Price"],
                                 mode="lines",
                                 name="Price"))

        # Plotting the moving average using a line plot
        fig.add_trace(go.Scatter(x=np.array(indicators.index),
                                 y=indicators["Moving Average"],
                                 mode="lines",
                                 name="MA"))
        
        # Plotting the Upper Band using a line plot 
        fig.add_trace(go.Scatter(x=np.array(indicators.index),
                                 y=indicators["Upper Band"],
                                 mode="lines",
                                 name="UB"))
        
        # Plotting the Lower Band using a line plot
        fig.add_trace(go.Scatter(x=np.array(indicators.index),
                                 y=indicators["Lower Band"],
                                 mode="lines",
                                 name="LB"))
        
        # Plotting the OHLC data using candlesticks
        fig.add_trace(go.Candlestick(x=np.array(ohlc.index),
                                     open=ohlc["Open"],
                                     high=ohlc["High"],
                                     low=ohlc["Low"],
                                     close=ohlc["Close"],
                                     name="OHLC"))
        
        # Updating the layout of the plot to remove the range slider and increase the height
        fig.update_layout(xaxis_rangeslider_visible=False, height=800)

        # Displaying the dreated figure to the screen
        fig.show()

In [19]:
def Threader(*instances, stop_time, logger):
    threads = []

    for instance in instances:
        thread = threading.Thread(target=instance.Bot, args=(stop_time, logger))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

In [21]:
AAPL_Bot = Trading(client, "AAPL", 21, 1, 5, 60)
AMZN_Bot = Trading(client, "AMZN", 21, 1, 3, 60)
MSFT_Bot = Trading(client, "MSFT", 21, 1, 3, 60)
NVDA_Bot = Trading(client, "NVDA", 21, 1, 3, 60)

Initialised
Initialised
Initialised
Initialised


In [33]:
Threader(AAPL_Bot, AMZN_Bot, MSFT_Bot, NVDA_Bot, stop_time="2024-01-12 20:30:00", logger=logging)

Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Bought
Bought
Sold
Sold
Sold
Sold
Sold
Strategy ERROR: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Sold
Bought
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Bought
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
SoldSold

Sold
Sold
Sold
Sold
Bought
Sold
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Bought
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Sold
Bought
Sold
Bought
Bought
Sold
S

In [39]:
AAPL_Bot.Plotting("1m")

In [35]:
AMZN_Bot.Plotting("2m")

In [36]:
MSFT_Bot.Plotting("2m")

In [32]:
NVDA_Bot.Plotting("2m")